In [1]:
class genere_distributions():
    x_min, x_max = 0., 10.
    y_min, y_max = 0., 10.
    nx, ny = 150, 150
    nx, ny = 200, 200
    offset = -0.1
    
    def __init__(self, mu, sigma, angle, prob_C):
        self.mu = mu
        self.sigma = sigma
        self.angle = angle
        self.prob_C = prob_C
        
        self.genere_grid()
        self.genere_pdf()
        self.colore_zone_influence()


    def genere_grid(self):
        self.xx, self.yy = np.meshgrid(np.linspace(self.x_min, self.x_max, self.nx), 
                                       np.linspace(self.y_min, self.y_max, self.ny))
        self.pos = np.dstack((self.xx, self.yy))
        


    # Modèles génératifs pour les distributions normales 2D 
    # (PDF: probability distribution functions)
    def genere_pdf(self):
        modeles = [None]*2
        for i in range(2):
            # Matrice de rotation
            theta = np.radians(self.angle[i])
            c, s = np.cos(theta), np.sin(theta)
            R = np.array(((c, -s), (s, c)))        

            # Matrice de covariance sans rotation
            C = np.array([[self.sigma[i, 0]**2, 0.],[0., self.sigma[i, 1]**2]])

            # Matrice de covariance après rotation
            # new_cov = rotation_matrix @ cov @ rotation_matrix.T
            C = R.dot( C.dot(R.T) )  

            # Génère modele pour PDF normale 2D
            modeles[i] = multivariate_normal(self.mu[i,:], C) 

        # Génère PDF individuelles
        pdf0 = self.prob_C[0]*modeles[0].pdf(self.pos)
        pdf1 = self.prob_C[1]*modeles[1].pdf(self.pos)

        # Masque indiquant la zone d'influence du modèle 1
        self.masque = pdf1 > pdf0

        # PDF globale
        self.pdf = pdf0 + pdf1


    

    def colore_zone_influence(self):    
        # Création de l'image de la PDF globale colorée selon la zone d'influence de chaque PDF individuelle  
        # Colormaps utilisées pour représenter les deux PDF
        v = np.array(cm.viridis.colors)
        g = np.array(cm.plasma.colors)

        # Normalisation de la PDF globale
        Z = 255*(self.pdf - np.min(self.pdf)) / (np.max(self.pdf) - np.min(self.pdf))
        Z = Z.astype(int)

        self.I = np.zeros((self.nx,self.ny,3))

        indx = np.argwhere(self.masque)
        A = Z[self.masque]
        for k, idx in enumerate(indx):
            #self.I[idx[0],idx[1],:] = g[A[k]]
            self.I[idx[0],idx[1],:] = np.array([0.2, 0.2, 0.2])

        indx = np.argwhere(~self.masque)
        B = Z[~self.masque]
        for k, idx in enumerate(indx):
            #self.I[idx[0],idx[1],:] = v[B[k]]
            self.I[idx[0],idx[1],:] = np.array([1.0, 0.2, 0.2])
            
        # plt.imshow(I)    

    
    def genere_surf_3D(self, ax):
        # Génération de la surface 3D en 2 couleurs identifiant les zones d'influence
        self.rstride, self.cstride = 5, 5

        s = ax.plot_surface(self.xx, self.yy, self.pdf, rstride=self.rstride, cstride=self.cstride, linewidth=.5, antialiased=True, color='b', edgecolors='k')       
        a1 = s.__dict__['_original_facecolor']
        b1 = s.__dict__['_facecolors']
        c1 = s.__dict__['_facecolors3d']
        
        s = ax.plot_surface(self.xx, self.yy, self.pdf, rstride=self.rstride, cstride=self.cstride, linewidth=.5, antialiased=True, color='r', edgecolors='k')
        a2 = s.__dict__['_original_facecolor']
        b2 = s.__dict__['_facecolors']
        c2 = s.__dict__['_facecolors3d']
        
        Lx = int(self.nx/self.rstride)
        Ly = int(self.ny/self.cstride)

        mask = resize(self.masque, (Lx,Ly), order=0)
        indx = np.argwhere(mask)
        idx = indx[:,0]*Lx + indx[:,1]

        a = a1
        b = b1
        c = c1
        for i in idx:
            a[i,:] = a2[i,:]
            b[i,:] = b2[i,:]
            c[i,:] = c2[i,:]
        s.__dict__['_original_facecolor'] = a
        s.__dict__['_facecolors'] = b
        s.__dict__['_facecolors3d'] = c


    
    

    # Affiche la fonction de distribution normale des probabilités en 3D et ses contours en 2D
    def affiche_PDF_avec_contours(self, contours_remplis=False, affiche_labels=True, affiche_tickmarks=True):

        fig = plt.figure(figsize = (13,10))
        ax = fig.gca(projection='3d')

        # Génération de la surface 3D en 2 couleurs identifiant les zones d'influence
        self.genere_surf_3D(ax)
    
        # Contours 2D remplis en dessous
        if (contours_remplis==True):
            cset = ax.contourf(self.xx, self.yy, self.pdf, zdir='z', offset=self.offset, cmap='viridis')
        else:
            cset = ax.contour(self.xx, self.yy, self.pdf, zdir='z', offset=self.offset, levels = 10, cmap='hot') 
            
        # Affiche frontières entre les zones d'influence
        ax.contour(self.xx, self.yy, self.masque, [0.5], offset=self.offset, linewidths=2., colors='white') 

        ax.set_zlim(self.offset,np.max(self.pdf))
        ax.view_init(20, -20)

        if (affiche_labels==True):
            ax.set_ylabel('$x_{2}$', fontsize=18)
            ax.xaxis.set_rotate_label(False)  
            ax.set_xlabel('$x_{1}$', rotation=10, fontsize=18)
            ax.zaxis.set_rotate_label(False)  
            ax.set_zlabel('$f(x_{1},x_{2})$', rotation=0, fontsize=18)

        if (affiche_tickmarks==False):
            # Enlève tickmarks
            ax.axes.get_xaxis().set_ticks([])
            ax.axes.get_yaxis().set_ticks([])
            ax.axes.get_zaxis().set_ticks([])

        plt.show()
    

